In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext, datasets, math
from tqdm import tqdm
from datasets import load_dataset

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## 1. Load data - Wiki Text

Using a imdb dataset which contains the text dataset of imdb.


In [5]:
dataset = load_dataset('imdb')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [7]:
print(dataset['train'].shape)

(25000, 2)


## 2. Preprocessing

### Tokenizing

Simply tokenize the given text to tokens.

In [8]:
# Import necessary libraries
import torchtext

# Define a basic English tokenizer using torchtext
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')

# Define a lambda function to tokenize the text in each example
# The lambda function takes an 'example' dictionary and a 'tokenizer'
# It returns a new dictionary with a 'tokens' key containing the tokenized text
tokenize_data = lambda example, tokenizer: {'tokens': tokenizer(example['text'])}

# Apply the tokenization function to each example in the dataset
# Remove the 'text' column and add a new 'tokens' column
# Use the specified tokenizer for tokenization
tokenized_dataset = dataset.map(tokenize_data, remove_columns=['text'], fn_kwargs={'tokenizer': tokenizer})


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [9]:
print(tokenized_dataset['train'][55]['tokens'])

['if', 'the', 'term', 'itself', 'were', 'not', 'geographically', 'and', 'semantically', 'meaningless', ',', 'one', 'might', 'well', 'refer', 'to', 'ned', 'kelly', 'as', 'an', 'australian', 'western', '.', 'for', 'the', 'people', 'down', 'under', ',', 'ned', 'kelly', 'was', ',', 'apparently', ',', 'a', 'folk', 'hero', 'bandit', 'akin', 'to', 'robin', 'hood', ',', 'jesse', 'james', ',', 'bonnie', 'and', 'clyde', ',', 'and', 'butch', 'cassidy', 'and', 'the', 'sundance', 'kid', '.', 'the', 'descendant', 'of', 'irish', 'immigrants', ',', 'kelly', 'became', 'a', 'fugitive', 'and', 'an', 'outlaw', 'after', 'he', 'was', 'falsely', 'accused', 'of', 'shooting', 'an', 'australian', 'law', 'officer', ',', 'a', 'crime', 'for', 'which', 'his', 'equally', 'innocent', 'mother', 'was', 'put', 'into', 'prison', '.', 'to', 'get', 'back', 'at', 'the', 'government', 'for', 'this', 'mistreatment', ',', 'kelly', ',', 'his', 'brother', 'dan', ',', 'and', 'two', 'other', 'companions', ',', 'became', 'notorious

### Numericalizing



In [10]:
# Build a vocabulary from the tokenized dataset for the 'train' split
# min_freq=3 means that tokens with frequency less than 3 will be ignored
vocab = torchtext.vocab.build_vocab_from_iterator(tokenized_dataset['train']['tokens'], min_freq=3)

# Insert special tokens '<unk>' and '<eos>' at specific indices in the vocabulary
vocab.insert_token('<unk>', 0)  # '<unk>' token is assigned index 0
vocab.insert_token('<eos>', 1)  # '<eos>' token is assigned index 1

# Set the default index for the vocabulary to the index of '<unk>'
vocab.set_default_index(vocab['<unk>'])


In [11]:
print(len(vocab))

40252


In [12]:
print(vocab.get_itos()[:10])

['<unk>', '<eos>', 'the', '.', ',', 'and', 'a', 'of', 'to', "'"]


## 3. Prepare the batch loader

### Prepare data


In [13]:
import torch

def get_data(dataset, vocab, batch_size):
    data = []

    # Iterate over examples in the dataset
    for example in dataset:
        # Check if the 'tokens' field is not empty
        if example['tokens']:
            # Append '<eos>' to the end of the token list
            tokens = example['tokens'] + ['<eos>']

            # Convert tokens to indices using the vocabulary
            # Note: Using a list comprehension to map each token to its index in the vocabulary
            tokens = [vocab[token] for token in tokens]

            # Extend the data list with the token indices
            data.extend(tokens)

    # Convert the data list to a PyTorch LongTensor
    data = torch.LongTensor(data)

    # Calculate the number of batches
    num_batches = data.shape[0] // batch_size

    # Trim the data to ensure it's a multiple of batch size
    data = data[:num_batches * batch_size]

    # Reshape the data tensor to have dimensions [batch_size, sequence_length]
    data = data.view(batch_size, num_batches)  # using view to reshape

    # Return the resulting data tensor
    return data  # [batch size, seq len]


In [14]:
batch_size = 128
train_data = get_data(tokenized_dataset['train'], vocab, batch_size)
valid_data = get_data(tokenized_dataset['unsupervised'], vocab, batch_size)
test_data  = get_data(tokenized_dataset['test'],  vocab, batch_size)

In [15]:
train_data.shape

torch.Size([128, 53063])

## 4. Modeling

<img src="figures/LM.png" width=600>

In [16]:
import torch
import torch.nn as nn
import math

class LSTMLanguageModel(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim, num_layers, dropout_rate):
        super().__init__()

        # Define model hyperparameters
        self.num_layers = num_layers
        self.hid_dim = hid_dim
        self.emb_dim = emb_dim

        # Define layers
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hid_dim, num_layers=num_layers, dropout=dropout_rate, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hid_dim, vocab_size)

        # Initialize weights
        self.init_weights()

    def init_weights(self):
        # Initialize embedding and linear layer weights with specific ranges
        init_range_emb = 0.1
        init_range_other = 1 / math.sqrt(self.hid_dim)
        self.embedding.weight.data.uniform_(-init_range_emb, init_range_other)
        self.fc.weight.data.uniform_(-init_range_other, init_range_other)
        self.fc.bias.data.zero_()

        # Initialize LSTM weights with specific ranges
        for i in range(self.num_layers):
            self.lstm.all_weights[i][0] = torch.FloatTensor(self.emb_dim, self.hid_dim).uniform_(-init_range_other, init_range_other)
            self.lstm.all_weights[i][1] = torch.FloatTensor(self.hid_dim, self.hid_dim).uniform_(-init_range_other, init_range_other)

    def init_hidden(self, batch_size, device):
        # Initialize hidden and cell states with zeros
        hidden = torch.zeros(self.num_layers, batch_size, self.hid_dim).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hid_dim).to(device)
        return hidden, cell

    def detach_hidden(self, hidden):
        # Detach hidden and cell states to avoid gradient computation
        hidden, cell = hidden
        hidden = hidden.detach()
        cell = cell.detach()
        return hidden, cell

    def forward(self, src, hidden):
        # Forward pass through the model
        embedding = self.dropout(self.embedding(src))
        output, hidden = self.lstm(embedding, hidden)
        output = self.dropout(output)
        prediction = self.fc(output)
        return prediction, hidden


## 5. Training



In [17]:
vocab_size = len(vocab)
emb_dim = 1024
hid_dim = 1024
num_layers = 2
dropout_rate = 0.65
lr = 1e-3

In [18]:
model      = LSTMLanguageModel(vocab_size, emb_dim, hid_dim, num_layers, dropout_rate).to(device)
optimizer  = optim.Adam(model.parameters(), lr=lr)
criterion  = nn.CrossEntropyLoss()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

The model has 99,269,948 trainable parameters


In [19]:
def get_batch(data, seq_len, idx):

    src    = data[:, idx:idx+seq_len]
    target = data[:, idx+1:idx+seq_len+1]
    return src, target

In [20]:
from tqdm import tqdm

def train(model, data, optimizer, criterion, batch_size, seq_len, clip, device):
    # Set the model to training mode
    model.train()

    # Initialize epoch loss
    epoch_loss = 0

    # Trim the data to have a length that is a multiple of seq_len
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches - 1) % seq_len]
    num_batches = data.shape[-1]

    # Initialize hidden state
    hidden = model.init_hidden(batch_size, device)

    # Iterate through batches using a sequence length (seq_len) stride
    for idx in tqdm(range(0, num_batches - 1, seq_len), desc='Training: ', leave=False):
        # Zero the gradients
        optimizer.zero_grad()

        # Detach hidden state to prevent gradient computation from previous sequences
        hidden = model.detach_hidden(hidden)

        # Get source and target batches
        src, target = get_batch(data, seq_len, idx)
        src, target = src.to(device), target.to(device)

        # Get batch size
        batch_size = src.shape[0]

        # Forward pass through the model
        prediction, hidden = model(src, hidden)

        # Reshape the prediction and target tensors for computing the loss
        prediction = prediction.reshape(batch_size * seq_len, -1)
        target = target.reshape(-1)

        # Calculate the loss
        loss = criterion(prediction, target)

        # Backward pass and optimization step
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        # Accumulate the epoch loss
        epoch_loss += loss.item() * seq_len

    # Return the average epoch loss
    return epoch_loss / num_batches


In [21]:
def evaluate(model, data, criterion, batch_size, seq_len, device):

    epoch_loss = 0
    model.eval()
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]
    num_batches = data.shape[-1]

    hidden = model.init_hidden(batch_size, device)

    with torch.no_grad():
        for idx in range(0, num_batches - 1, seq_len):
            hidden = model.detach_hidden(hidden)
            src, target = get_batch(data, seq_len, idx)
            src, target = src.to(device), target.to(device)
            batch_size= src.shape[0]

            prediction, hidden = model(src, hidden)
            prediction = prediction.reshape(batch_size * seq_len, -1)
            target = target.reshape(-1)

            loss = criterion(prediction, target)
            epoch_loss += loss.item() * seq_len
    return epoch_loss / num_batches

Even if the pc is integrated with gpu, every steps used for setting up CUDA, this was very time consuming. After some time, the session was timed out, so could only do 1 epoch for this.

In [22]:
import torch.optim as optim
import math

n_epochs = 1
seq_len = 50
clip = 0.25

# Define a learning rate scheduler to adjust the learning rate during training
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0)

# Initialize the best validation loss to positive infinity
best_valid_loss = float('inf')

# Loop through epochs
for epoch in range(n_epochs):
    # Train the model and calculate the training loss
    train_loss = train(model, train_data, optimizer, criterion, batch_size, seq_len, clip, device)

    # Evaluate the model on the validation set and calculate the validation loss
    valid_loss = evaluate(model, valid_data, criterion, batch_size, seq_len, device)

    # Adjust the learning rate based on the validation loss
    lr_scheduler.step(valid_loss)

    # Save the model if the validation loss improves
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'best-val-lstm_lm.pt')

    # Print the training and validation perplexities for the current epoch
    print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
    print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')


	Train Perplexity: 407.294
	Valid Perplexity: 188.212


## 6. Testing

In [23]:
model.load_state_dict(torch.load('best-val-lstm_lm.pt',  map_location=device))
test_loss = evaluate(model, test_data, criterion, batch_size, seq_len, device)
print(f'Test Perplexity: {math.exp(test_loss):.3f}')

Test Perplexity: 185.220


## 7. Real-world inference



In [26]:
def generate(prompt, max_seq_len, temperature, model, tokenizer, vocab, device, seed=None):
    # Set a random seed if provided
    if seed is not None:
        torch.manual_seed(seed)

    # Set the model to evaluation mode
    model.eval()

    # Tokenize the input prompt and convert to indices using the vocabulary
    tokens = tokenizer(prompt)
    indices = [vocab[t] for t in tokens]

    # Set batch size to 1 for generating one sequence at a time
    batch_size = 1

    # Initialize hidden state for the model
    hidden = model.init_hidden(batch_size, device)

    # Disable gradient computation during generation
    with torch.no_grad():
        # Generate text up to the specified maximum sequence length
        for i in range(max_seq_len):
            # Convert the current sequence to a PyTorch tensor and move to the specified device
            src = torch.LongTensor([indices]).to(device)

            # Forward pass through the model to get the next prediction and update hidden state
            prediction, hidden = model(src, hidden)

            # Apply temperature to the last prediction and sample from the distribution
            probs = torch.softmax(prediction[:, -1] / temperature, dim=-1)
            prediction = torch.multinomial(probs, num_samples=1).item()

            # Ensure the generated token is not '<unk>'
            while prediction == vocab['<unk>']:
                prediction = torch.multinomial(probs, num_samples=1).item()

            # Break if the generated token is '<eos>'
            if prediction == vocab['<eos>']:
                break

            # Append the generated token index to the sequence
            indices.append(prediction)

    # Convert the generated indices back to tokens using the vocabulary
    itos = vocab.get_itos()
    tokens = [itos[i] for i in indices]

    # Return the generated tokens
    return tokens


In [28]:
prompt = ' Harry Potter is'
max_seq_len = 20
seed = 0


temperatures = [0.5, 0.7, 0.75, 0.8, 1.0]
for temperature in temperatures:
    generation = generate(prompt, max_seq_len, temperature, model, tokenizer,
                          vocab, device, seed)
    print(str(temperature)+'\n'+' '.join(generation)+'\n')

0.5
harry potter is a very good character . i am a very good job . the movie is also the most part of

0.7
harry potter is on a local man who is other to the best . he ' s also the most judy recruits ,

0.75
harry potter is on a local man who is other to the best . he ' s also the nonsense of the proper

0.8
harry potter is on a local man who is other to the best . he ' s also the nonsense of the proper

1.0
harry potter is on a local menace , but i suspect other peoples around their directors and the men are the nonsense judy



In [29]:
torch.save(model.state_dict(), '/content/bestmodel.pth')

In [30]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenizer.save_pretrained('content/tokenizer')


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

('content/tokenizer/tokenizer_config.json',
 'content/tokenizer/special_tokens_map.json',
 'content/tokenizer/vocab.txt',
 'content/tokenizer/added_tokens.json',
 'content/tokenizer/tokenizer.json')